Let us setup an interactive `Python` session where we can run the code
developed here.

``` jupyter
print("Welcome to EMACS Jupyter")
```

We have an extensive configuration of the ususal 240 `flatmap-columns`
that span the entire SSCx. There is an alternate version that has been
used for several simulations. It is expected to cover the central area
of the SSCx `flatmap`. Here we will develop a `connsense-pipeline` to
compute the same analyses as the `flatmap-topology` pipeline. The only
difference is the input flatmap columns, which are now (\<2023-03-31
Fri\>) available as a dataframe.

To test the configuration, we will need to setup a `Python` environment,

# Setup

To get the notebook you will have to clone,

``` shell
git clone https://bbpgitlab.epfl.ch/conn/structural/topological-analysis-of-subvolumes.git
git checkout beta
```

In our discussion we will develop scientific concepts to measure the
circuit, and implement Python functions to compute them. Here we setup a
notebook template to test and explore, and the structure of a `Python`
package for our methods.

In [ ]:
from importlib import reload
from collections.abc import Mapping
from collections import OrderedDict
from pprint import pprint, pformat
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib

reload(matplotlib)
from matplotlib import pylab as plt
import seaborn as sbn

from IPython.display import display

from bluepy import Synapse, Cell, Circuit

GOLDEN = (1. + np.sqrt(5.))/2.
print("We will plot golden aspect ratios: ", GOLDEN)


## Workspaces

We have run `connsense-CRAP` for the SSCx dissemination variant *Bio-M*,
extracting data that we will use to compute the factology. Here is a
list of workspaces we will need to generate factsheets.

In [ ]:
ROOTSPACE = Path("/")
PROJSPACE = ROOTSPACE / "gpfs/bbp.cscs.ch/project/proj83"
SOODSPACE = PROJSPACE / "home/sood"
HEXTAP = SOODSPACE / "topological-analysis-subvolumes/test/v2/hexes"
DEVSPACE  = HEXTAP / "test" / "develop"


## `connsense` Modules

While test-developing it will be good to have direct access to the
`connsense-TAP-store` we will use. We will use a module from `connsense`
to load the HDFstore,

In [ ]:
from connsense.develop import topotap as cnstap
hextap = cnstap.HDFStore(HEXTAP/"pipeline.yaml")
circuit = hextap.get_circuit("Bio_M")
print("Available analyses in topology tap: ")
pprint(hextap.analyses)
circuit


# paths

We need to set paths to the artefacts that the pipeline will use. We set
paths for the circuit to analyze, and the root space for the pipeline's
HDF5 stores, and the HDF5 group for each of the pipeline steps. The only
change we need to make here, building on the config we alread have for
`flatmap-columns`, is to set the `pipeline.root`,

``` yaml
paths:
  description: >-
    The ~connsense~ pipeline needs paths to the input data to load from, and output paths to store data.
    Paths to the circuit must be provided along with paths to the HDF5 archive that will store the pipeline's
    results.
  format: relative
  circuit:
    root: "/gpfs/bbp.cscs.ch/project/proj83/circuits"
    files:
      Bio_M: "Bio_M/20200805/CircuitConfig_TC_WM"
  pipeline:
    root: "/gpfs/bbp.cscs.ch/project/proj83/home/sood/topological-analysis-subvolumes/test/v2/hexes"
    input:
      store: "connsense.h5"
    output:
      store: "connsense.h5"
    steps:
      define-subtargets: "subtargets"
      extract-node-populations: "nodes/populations"
      extract-edge-populations: "edges/populations"
      analyze-connectivity: "analyses/connectivity"
```

# parameters

We provide the parameters for each step.

``` yaml
parameters:
  description: >-
    Provide parameters that apply to each step.
```

The pipeline will `define` the `subtargets`, `extract` circuit artefacts
for the `subtargets`, and `analyze` the results. Each of these `steps`
need to be defined.

## define-subtargets

``` yaml
define-subtargets:
  description: >-
    Configure the subtargets to analyze, entrying each definition as a key, value.
  definitions:
```

We define `flatmap-columns` which are prism like columns orientated
along cortical layers, from white-matter to pia. The data is loaded from
an `NRRD` that maps each circuit voxel to an `id` that corresponds to
`flatmap-column-subtarget`. We also need `info`, a `dataframe` that
provides information about these `subtargets` indexing them by the `id`
from the `NRRD`.

### hexes

``` yaml
hexes:
  description: >-
    Columns like the flatmap columns, based on hexagonal tiling.
  input:
    circuit:
    - "Bio_M"
  kwargs:
    path: "/gpfs/bbp.cscs.ch/project/proj83/home/sood/topological-analysis-subvolumes/test/v2/hexes/hexes.csv"
  loader:
    source: "/gpfs/bbp.cscs.ch/project/proj83/home/sood/topological-analysis-subvolumes/test/v2/hexes/topology/pandas.py"
    method: "read_csv"
```

The `input` is set to a label that should appear among `config-paths`.
The `loader` method used is expected to take a `bluepy.Circuit` instance
as an argument, which will be passed by `connsense`.

We need to implement the `read_csv` method to load the `hexmap-columns`,

In [ ]:
def read_csv(path):
    """..."""
    import pandas as pd
    subtargets_annotation = pd.read_csv(path)
    subtarget_ids = subtargets_annotation.subtarget_id
    subtargets = (pd.Index(subtarget_ids.unique(), name="subtarget_id").to_series()
                  .apply("Hex{}".format))
    circuit_gidses = subtargets_annotation.groupby("subtarget_id").gid.apply(list)
    return (subtargets, None, circuit_gidses)


We have already setup the paths to load the `Python` code above from
\~connsense-.

## create-index

Within `connsense` we use an `internal` index for
`subtarget, circuit, connectome`. While we can infer this information
from <span class="spurious-link" target="paths">*paths*</span> and <span
class="spurious-link"
target="define-subtargets">*define-subtargets*</span>, we have not yet
(\<2023-02-13 Mon\>) implemented this feature.

``` yaml
create-index:
  description:
    Create tap-store indices by listing datasets for each index variable.
  variables:
    circuit:
      - "Bio_M"
    connectome:
      - "local"
      - "intra_SSCX_midrange_wm"
      - "Thalamocortical_input_VPM"
      - "Thalamocortical_input_POM"
    subtarget:
      dataset: ["define-subtargets", "hexes/name"]
```

## extract-node-populations

We will extract populations of nodes, naming them as we would like to in
our analyses to follow.

``` yaml
extract-node-populations:
  description: >-
    Specify the populations to extract from a circuit.
  populations:
```

### default

The population of *non-barrel* SSCx biophysical nodes will be the
node-population that we analyze,

``` yaml
default:
  description: >-
    The default population will be that of neurons in the SSCx.
    To extract the neurons we will use a `connsense` method that uses ~bluepy~.
  input:
    subtarget:
      dataset:  ["define-subtargets", "hexes"]
    circuit:
      - "Bio_M"
  kwargs:
    properties:
      - region
      - layer
      - x
      - y
      - z
      - depth
      - synapse_class
      - mtype
      - etype
      - morphology
  extractor:
    source: connsense.extract_nodes.bluepy
    method: extract_node_properties
  output: "pandas.DataFrame"
```

## extract-edge-populations

Just as for `node-populations` we will extract `edge-populations`.
Notice that the paths we have set in the `tap-HDFStore` are motivated by
equivalent terminology in SONATA.

``` yaml
extract-edge-populations:
  description: >-
    Specify the edge populations to extract from a circuit.
  populations:
```

We will analyze topology of the `local` connectome that consists of the
connections among cells based on their axo-dendritic appositions. To
study the local connectivity of a `subtarget`, we will need it's
adjacency matrix that we can extract to the `TAPStore`.

### local

The population of local connections resulting from axo-dendritic
appositions,

``` yaml
local:
  input:
    subtarget:
      dataset:  ["define-subtargets", "hexes"]
    circuit:
      - "Bio_M"
    connectome:
      - "local"
  extractor:
    source: micasa.connsense.develop.extract.edge_populations.extract_connectivity
    method: extract_adj
  output: "sparse.spmatrix"
```

Normally extraction of edges will be a slow step.

### long-range

We can extract *long-rage* connectivity *between* `flatmap-columns` by
defining computations of pairs of `flatmap-columns`.

Our initial goal of extracting `long-range` connectivity is to
investigate the topolgical structure of simplices whose
`source-vertex lies in a selected ~flatmap-columns`. The
`source-subtarget` `flatmap-columns` that we will be interested in will
be determined by their `innervation` from thalamo-cortical projections.

We will extract two different `long-range` edge-populations. The
edge-population `long-range-sources` (of the `flatmap-columns`) will
contain a `list(adjacency-matrix)` for each `flatmap-column`. For a
given `flatmap-column` `X`, there will be one `adjacency-matrix` for
each of another `flatmap-column` `Y`. The `adjacency-matrix` for `X, Y`
will contain edges that have their `targets` in `X`, and `sources` in
`Y`.

In [ ]:
def long_range_sources(flatmap_column, source_flatmap_column=None):
    """Long range sources of a flatmap-column."""
    dataset = cnstap.TapDataset(tap, ("extract-edge-populations", "long-range-sources"))
    matrices = dataset.loc[flatmap_column, 0, 0]
    return matrices if not source_flatmap_column else matrices.loc[source_flatmap_column]


will give us a `pandas.Series(adjacency-matrix)`, one for each
`flatmap-column` among a selection of `sources`.

For `long-range-targets`,

In [ ]:
def long_range_targets(flatmap_column, target_flatmap_column=None):
    """Long range targets of a flatmap-column."""
    dataset = cnstap.TapDataset(tap, ("extract-edge-populations", "long-range-targets"))
    matrices = dataset.loc[flatmap_column, 0, 0]
    return matrices if not target_flatmap_column else matrices.loc[target_flatmap_column]


The population `long-range-sources` will be of edges (represented as
(CSR) directed adjacency matrices), one for each pair `(X, Y)` of
`flatmap-columns`. The `edge-population` for a `flatmap-column` `X` will
have it's `targets` among nodes in `X`, while the `sources` of the edges
will be among another. The primary index in `connsense` will be the
`flatmap-column` that is receiving the `incoming` conenctions. With
`circuit, connectome` specified, we will have another `input`, the
`sources` that will specify a subset of `flatmap-columns` to be
*crossed* with each dataset in `connsense-TAP` that matches the
specification of `subtarget, circuit, connectome` inputs.

Well, finally we decided to keep things explicit, with a `pre-` and a
\~post- specified as input arguments to the method that will extract the
cross adjacency,

``` yaml
long-range-cross:
  input:
    pre:
      dataset: ["define-subtargets", "flatmap-columns"]
      subset: [[100, 0]]
    circuit:
      - "Bio_M"
    connectome:
      - "local"
    post:
      dataset: ["define-subtargets", "hexes"]
    join: CROSS
  computation:
    source: micasa.connsense.develop.extract.edge_populations.extract_connectivity
    method: extract_cross_adj
  output: "sparse.spmatrix"
```

## analyze-connectivity

We will analyze several phenomena related to network topology, each
entered in `analyses`,

``` yaml
analyze-connectivity:
  description: >-
    Configure each analyses' parameters, as a mapping under section `analyses`.
  analyses:
```

### simplex-counts

We count the number of *simplices* that is complete subgraphs of a given
dimension *i.e* the number of edges in the simplex. So a node is a
simplex of dimension 0, an edge of dimension 1 while a dimension 2 will
be a triangle. We will compute a series of simplex counts by dimension
for each subtarget, 5 of it's Erdos-Renyi controls, subgraphs in each
layer and their ER controls.

1.  description

    ``` yaml
    simplex-counts:
      description: >-
        Number of simplices by dimension.
    ```

2.  input

    We will compute `simplex-counts` for each of the `flatmap-columns`,
    using `adjacency` matrices that we extract as dataset
    `("extract-edge-populations", "local")`. We will also use
    `node_properties` that we extract as `node-population` `default`.

    ``` yaml
    input:
      node_properties:
        dataset: ["extract-node-populations", "default"]
      adjacency:
        dataset: ["extract-edge-populations", "local"]
    ```

3.  controls

    We will use random controls for each `subtarget`, entering them by
    name and value. We start with Erdos-Renyi controls, 5 of them
    specified by seed.

    ``` yaml
    controls:
      erdos-renyi:
        description: >-
          Erdos-Renyi shuffle of edges.
        seeds: [0, 1, 2, 3, 4]
        algorithm:
          source: "/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/proj83/connectome_analysis/library/randomization.py"
          method: "ER_shuffle"
    ```

4.  slicing

    We will slice each `subtarget` into `subtgraphs` consisting of each
    of the cortical layers. Thus a single `subtarget` should give us 6
    of these `slicings`. To compute analysis on a `slicing` we will have
    to enter it inside the `analysis` config.

    We may store computation of each slice of a single subtarget as a
    `datacall`. This will result in each slice of each subtarget to be
    sequenced as a `unit-computation`. In `connsense-parallelization`,
    we then parallelize based on the sizes of all the slices. The
    computation is seprated from that of `full-subtargets`. This does
    not work well with `simplex-counts` as `datacall` of a single
    `slice` requires us to first load the adjacency matrices, then
    control them. With large adjacency matrices the computation of a
    unit `slice-subtarget` will require the same resources as loading
    the original full and randomizing it. Instead we will have to
    serially compute the slices' simplex-counts for each subtarget.

    We can specify this in the config as `compute_mode: EXECUTE`,
    instead of `DATACALL` which will create individual `datacalls`, or
    `DATASET` that should save each `slice-subtarget` to a `TapDataset`.

    ``` yaml
    slicing:
      description: >-
        Slice analysis input according to some rules.
      do-full: true #to run the original full matrices as well...
      layer:
        description: >-
          Intralayer subgraphs.
        compute_mode: EXECUTE
        slices:
          layer: [1, 2, 3, 4, 5, 6]
        algorithm:
          source: "/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/proj83/connectome_analysis/library/topology.py"
          method: "subgraph_intralayer"
    ```

5.  computation

    Finally, we need to specify the method to use to compute
    `simplex-counts` for each `subtarget`, and the data-type of it's
    output.

    ``` yaml
    computation:
      source: "/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/proj83/connectome_analysis/library/topology.py"
      method: "simplex_counts"
    output: "pandas.Series"
    ```

### simplex-source-count of cross connectivity

A simplex `s` is *efferent* to a node `n` if each of it's vertices is
*efferently* connected to `n`. Given a `flatmap-column~s ~X` and `Y`, we
want to count for each node `x` of `X` the number of simplices in `y`
that are *efferent* to `x`.

1.  description

    ``` yaml
    long-range-simplex-sources:
      description: >-
        For each node N in flatmap-column X, count the simplices in flatmap-column Y
        that are efferent to N
    ```

2.  input

    We will compute `simplex-counts` for each of the `flatmap-columns`,
    using `adjacency` matrices that we extract as dataset
    `("extract-edge-populations", "local")`. We will also use
    `node_properties` that we extract as `node-population` `default`.

    ``` yaml
    input:
      xadj:
        dataset: ["extract-edge-populations", "long-range/sources"]
      node_properties:
        datajoin:
          pre:
            dataset: ["extract-node-populations", "default"]
          post:
            dataset: ["extract-node-populations", "default"]
      adj:
        datajoin:
          pre:
            dataset: ["extract-edge-populations", "local"]
    ```

3.  computation

    Finally, we need to specify the method to use to compute
    `simplex-counts` for each `subtarget`, and the data-type of it's
    output.

    ``` yaml
    computation:
      source: "/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/proj83/connectome_analysis/library/topology.py"
      method: "simplex_counts"
    output: "pandas.Series"
    ```

### model-params-dd2

This analysis is used to create parameters for the distance dependent
connection-probablity order 2 control model.

``` yaml
model-params-dd2:
  description: >-
    Parameters for distance dependent connectivity model of order 2.
    Note that the `coord_names` in key `kwargs:` must agree with the configuration
    of the control model that will use the results of this analysis.
```

1.  input

    We will compute `simplex-counts` for each of the `flatmap-columns`,
    using `adjacency` matrices that we extract as dataset
    `("extract-edge-populations", "local")`. We will also use
    `node_properties` that we extract as `node-population` `default`.

    ``` yaml
    input:
      node_properties:
        dataset: ["extract-node-populations", "default"]
      adjacency:
        dataset: ["extract-edge-populations", "local"]
    ```

2.  computation

    Finally, we need to specify the method to use to compute
    `simplex-counts` for each `subtarget`, and the data-type of it's
    output.

    ``` yaml
    computation:
      source: "/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/proj83/connectome_analysis/library/modelling.py"
      method: "conn_prob_2nd_order_model"
    output: "pandas.DataFrame"
    ```

3.  kwargs

    The method that we will use to compute model parameters needs these
    parameters,

    ``` yaml
    kwargs:
      bin_size_um: 50
      max_range_um: 1000
      sample_size: null
      coord_names: ["x", "y", "z"]
    ```

### thalamic-innervation

We will compute the number of afferent-synapses on each neuron in a
`subtarget` from the *thalamo-cortical* projections.

1.  description

    ``` yaml
    thalamic-innervation:
      description: >-
        Number of thalamo-cortical synapses at each cell in a subtarget.
    ```

2.  vpm:

    1.  input

        We will compute `simplex-counts` for each of the
        `flatmap-columns`, using `adjacency` matrices that we extract as
        dataset `("extract-edge-populations", "local")`. We will also
        use `node_properties` that we extract as `node-population`
        `default`.

        ``` yaml
        vpm:
          description: >-
            Thalamic innervation of a subtarget, that originates in the VPM.
          input:
            subtarget:
              dataset: ["define-subtargets", "flatmap-columns"]
            circuit:
              - "Bio_M"
            connectome:
              - "Thalamocortical_input_VPM"
        ```

    2.  computation

        Finally, we need to specify the method to use to compute
        `simplex-counts` for each `subtarget`, and the data-type of it's
        output.

        ``` yaml
        computation:
          source: micasa.connsense.develop.analyze.composition.projections.projections
          method: "innervate_cells"
        output: "pandas.Series"
        ```

3.  pom:

    1.  input

        We will compute `simplex-counts` for each of the
        `flatmap-columns`, using `adjacency` matrices that we extract as
        dataset `("extract-edge-populations", "local")`. We will also
        use `node_properties` that we extract as `node-population`
        `default`.

        ``` yaml
        pom:
          description: >-
            Thalamic innervation of a subtarget, that originates in the VPM.
          input:
            subtarget:
              dataset: ["define-subtargets", "flatmap-columns"]
            circuit:
              - "Bio_M"
            connectome:
              - "Thalamocortical_input_POM"
        ```

    2.  computation

        Finally, we need to specify the method to use to compute
        `simplex-counts` for each `subtarget`, and the data-type of it's
        output.

        ``` yaml
        computation:
          source: micasa.connsense.develop.analyze.composition.projections.projections
          method: "innervate_cells"
        output: "pandas.Series"
        ```

### cross-col-k-indegree

``` yaml
description: >-
  Compute generalized in-degree of nodes in adj_target from nodes in adj_source.
  The k-in-degree of a node v is the number of k-simplices in adj_source with all its nodes mapping to v
  through edges in adj_cross
input:
  adj_cross:
    dataset: ["extract-edge-populations", "long-range-cross"]
  adj_pre:
    dataset: ["extract-edge-populations", "local"]
    join_index:
      subtarget_id: pre_subtarget_id
      circuit_id: circuit_id
      connectome_id: DROP
```

# tap environment

With `connsense` we get the `tap` command line interface command that we
can use to setup and run a pipeline. For example, we can change
directory to the working space for `hexes` topology we are developing
here. We place the pipeline configuration develop above, along with the
`runtime` config there, and

``` shell
tap init
```

To formally setup a `workspace`. We can now start running the pipeline
step by step.

## define-subtargets

Before running any analyses we need to define the circuit's subtargets
that we want to apply our analyses to. We have already configured the
`hexes` as our `subtargets` above in <span class="spurious-link"
target="parameters">*parameters*</span>. We can get `connsense` to
define them in the `tap-store` from CLI,

``` shell
tap init define-subtargets hexes
```

Once initialized, we can directly run this step. We expect definition of
`subtargets` to a light-weight computation, and `connsense` does not
parallelize — as the quantity to parallelize over is defined in this
step. To run the step,

``` shell
tap run define-subtargets hexes
```

Now we will have the subtargets in `hextap`,

In [ ]:
hextap.subtarget_gids


We have also *named* the subtargets,

In [ ]:
hextap.subtargets


## extract-node-populations

For a pipeline step that will be parallelized, we need to setup it's
execution,

``` shell
tap setup extract-node-popuations default
```

To run we will have to launch the launchscript. We need to change to the
directory where the computation has been setup. Assuming that the
current working directory is our `hexes` workspace,

``` shell
pushd run/extract-node-popuations/default
source launchscript.sh
```

We will provide a direct `tap-command` to launch the jobs from CLI. Once
all the individual compute-nodes are done running, we can collect the
results,

``` shell
popd
tap collect extract-node-populations default
```

The result,

In [ ]:
hextap.nodes.frame


where each entry contains a dataframe of node-properties (as configured
in <span class="spurious-link" target="parameters">*parameters*</span>),

In [ ]:
hextap.nodes.frame.iloc[0]()


We will be using `setup, launch, collect` for each of the computations
that we have configured.